# Training With Transfer Learning
It's time to train the CNN. We will be using transfer learning - we'll take an existing, pretrained model and replace the fully connected layer. The model we'll be using is resnet50 model

We will train this model using mini-batch gradient descent and allow fine-tuning (allowing the bottleneck layers to be updated with each iteration).

In this notebook, we use PyTorch to train the model.

In [ ]:

# result_df[result_df['label'] == 'fish'][['label', 'guessed']]
# result_df[result_df['label'] == 'pinto_beans'][['label', 'guessed']]
# result_df[result_df['label'] == 'parmesan_cheese'][['label', 'guessed']]

# def show_result(result_df, item):
#     for i, row in result_df[result_df['label'] == item].iterrows():
#         imshow(row['image'], title="Label: {}, Guessed: {}".format(row['label'], row['guessed']), pause=2.5)

# for item in group_accuracy.index[:8]:
#     show_result(result_df, item)


# for _, row in result_df.iterrows():
#     if not row['correct']:
#         imshow(row['image'], 
#             title="Label: {}, Guessed: {}".format(row['label'], row['guessed']), pause=2.5) 


# show_result(result_df, 'beef')
# show_result(result_df, 'pork')
# show_result(result_df, 'brown_onion')
# show_result(result_df, 'chicken_leg')
# show_result(result_df, 'mushroom')
# show_result(result_df, 'cilantro')

